## Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tensorflow import keras

from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

In [2]:
pip install yfinance

In [3]:
import yfinance as yahooFinance

## Load Data

In [4]:

GetFacebookInformation = yahooFinance.Ticker("META")
df = GetFacebookInformation.history(period="max")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3169 entries, 2012-05-18 00:00:00-04:00 to 2024-12-20 00:00:00-05:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3169 non-null   float64
 1   High          3169 non-null   float64
 2   Low           3169 non-null   float64
 3   Close         3169 non-null   float64
 4   Volume        3169 non-null   int64  
 5   Dividends     3169 non-null   float64
 6   Stock Splits  3169 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 198.1 KB


In [6]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-05-18 00:00:00-04:00,41.889986,44.828761,37.855398,38.084522,573576400,0.0,0.0
2012-05-21 00:00:00-04:00,36.390988,36.520494,32.874422,33.900501,168192700,0.0,0.0
2012-05-22 00:00:00-04:00,32.485908,33.462179,30.822263,30.882034,101786600,0.0,0.0
2012-05-23 00:00:00-04:00,31.250625,32.376325,31.240663,31.878227,73600000,0.0,0.0
2012-05-24 00:00:00-04:00,32.824611,33.083620,31.649101,32.904305,50237200,0.0,0.0
...,...,...,...,...,...,...,...
2024-12-16 00:00:00-05:00,629.979980,631.150024,618.580017,624.239990,10885600,0.5,0.0
2024-12-17 00:00:00-05:00,626.169983,632.369995,616.539978,619.440002,12897800,0.0,0.0
2024-12-18 00:00:00-05:00,621.000000,627.359985,596.320007,597.190002,17075500,0.0,0.0


## EDA

In [7]:

df = df.reset_index()

In [8]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-05-18 00:00:00-04:00,41.889986,44.828761,37.855398,38.084522,573576400,0.0,0.0
1,2012-05-21 00:00:00-04:00,36.390988,36.520494,32.874422,33.900501,168192700,0.0,0.0
2,2012-05-22 00:00:00-04:00,32.485908,33.462179,30.822263,30.882034,101786600,0.0,0.0
3,2012-05-23 00:00:00-04:00,31.250625,32.376325,31.240663,31.878227,73600000,0.0,0.0
4,2012-05-24 00:00:00-04:00,32.824611,33.083620,31.649101,32.904305,50237200,0.0,0.0
...,...,...,...,...,...,...,...,...
3164,2024-12-16 00:00:00-05:00,629.979980,631.150024,618.580017,624.239990,10885600,0.5,0.0
3165,2024-12-17 00:00:00-05:00,626.169983,632.369995,616.539978,619.440002,12897800,0.0,0.0
3166,2024-12-18 00:00:00-05:00,621.000000,627.359985,596.320007,597.190002,17075500,0.0,0.0
3167,2024-12-19 00:00:00-05:00,610.390015,611.520020,595.000000,595.570007,14958100,0.0,0.0


In [9]:

fig1 = px.line(df, x='Date', y='Close', title='Facebook Stock Price Over Time')
fig1.show()

In [10]:

fig4 = go.Figure(data=[go.Candlestick(x=df['Date'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'])])

fig4.update_layout(title='Facebook Stock Prices - Candlestick Chart',
                   xaxis_title='Date',
                   yaxis_title='Stock Price')

fig4.show()

## Data preprocessing

In [11]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-05-18 00:00:00-04:00,41.889986,44.828761,37.855398,38.084522,573576400,0.0,0.0
1,2012-05-21 00:00:00-04:00,36.390988,36.520494,32.874422,33.900501,168192700,0.0,0.0
2,2012-05-22 00:00:00-04:00,32.485908,33.462179,30.822263,30.882034,101786600,0.0,0.0
3,2012-05-23 00:00:00-04:00,31.250625,32.376325,31.240663,31.878227,73600000,0.0,0.0
4,2012-05-24 00:00:00-04:00,32.824611,33.083620,31.649101,32.904305,50237200,0.0,0.0
...,...,...,...,...,...,...,...,...
3164,2024-12-16 00:00:00-05:00,629.979980,631.150024,618.580017,624.239990,10885600,0.5,0.0
3165,2024-12-17 00:00:00-05:00,626.169983,632.369995,616.539978,619.440002,12897800,0.0,0.0
3166,2024-12-18 00:00:00-05:00,621.000000,627.359985,596.320007,597.190002,17075500,0.0,0.0
3167,2024-12-19 00:00:00-05:00,610.390015,611.520020,595.000000,595.570007,14958100,0.0,0.0


In [12]:
train = df.iloc[:int(len(df)*0.9), :]

In [13]:
test = df.iloc[int(len(df)*0.9):, :]

In [14]:
training_processed = train.iloc[:, 4:5].values
training_processed

array([[ 38.08452225],
       [ 33.90050125],
       [ 30.8820343 ],
       ...,
       [299.16720581],
       [301.39868164],
       [303.90908813]])

In [15]:
testing_processed = test.iloc[:, 4:5].values

In [16]:
scaler = MinMaxScaler(feature_range = (0, 1))

training_scaled = scaler.fit_transform(training_processed)
testing_scaled = scaler.fit_transform(testing_processed)


In [17]:
training_scaled.shape

(2852, 1)

In [18]:
features_set = []
labels = []
for i in range(60, training_scaled.shape[0]):
    features_set.append(training_scaled[i-60:i, 0])
    labels.append(training_scaled[i, 0])

In [19]:
test_features_set = []
test_labels = []
for i in range(60, testing_scaled.shape[0]):
    test_features_set.append(testing_scaled[i-60:i, 0])
    test_labels.append(testing_scaled[i, 0])

In [20]:
len(features_set)

2792

In [21]:
len(labels)

2792

In [22]:
features_set, labels = np.array(features_set), np.array(labels)

In [23]:
test_features_set, test_labels = np.array(test_features_set), np.array(test_labels)

In [24]:
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
print(features_set.shape)

(2792, 60, 1)


In [25]:
test_features_set

array([[0.03269455, 0.02131496, 0.03099029, ..., 0.10666151, 0.1324822 ,
        0.13398412],
       [0.02131496, 0.03099029, 0.03604477, ..., 0.1324822 , 0.13398412,
        0.12944971],
       [0.03099029, 0.03604477, 0.03064372, ..., 0.13398412, 0.12944971,
        0.13450401],
       ...,
       [0.79909947, 0.81352598, 0.81216466, ..., 0.96428109, 0.97700883,
        0.96309248],
       [0.81352598, 0.81216466, 0.81077396, ..., 0.97700883, 0.96309248,
        0.89858426],
       [0.81216466, 0.81077396, 0.82549032, ..., 0.96309248, 0.89858426,
        0.8938875 ]])

In [26]:
test_features_set = np.reshape(test_features_set, (test_features_set.shape[0], test_features_set.shape[1], 1))
print(test_features_set.shape)

(257, 60, 1)


## RNN

#### Build RNN Model

In [27]:
RNN_model = Sequential()
RNN_model.add(SimpleRNN(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100))
RNN_model.add(Dropout(0.2))

RNN_model.add(Dense(units = 1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Compile RNN Model

In [28]:
RNN_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [29]:
RNN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 60, 300)             │          90,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 300)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 60, 100)             │          40,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 60, 100)             │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 171,001 (667.97 KB)

 Trainable params: 171,001 (667.97 KB)

 Non-trainable params: 0 (0.00 B)

#### Fit RNN Model


In [30]:
RNN_History = RNN_model.fit(features_set, labels, epochs = 20, batch_size = 32)

Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 31s 163ms/step - loss: 0.6050
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 19s 44ms/step - loss: 0.2838
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.2244
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.1681
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - loss: 0.1430
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.1202
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.1107
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 0.0985
Epoch 9/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0953
Epoch 10/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0870
Epoch 11/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0758
Epoch 12/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 0.0700
Epoch 13/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0698
Epoch 14/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0620
Epoch 15/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0512
E

## LSTM

####  Build LSTM Model

In [31]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(Dense(units = 1))

#### Compil LSTM Model

In [32]:
LSTM_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [33]:
LSTM_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 300)             │         362,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 60, 300)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 60, 100)             │         160,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 60, 100)             │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,701 (2.61 MB)

 Trainable params: 683,701 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

#### Fit LSTM Model

In [34]:
LSTM_History = LSTM_model.fit(features_set, labels, epochs = 20, batch_size = 32)

Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0362
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0030
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0022
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0019
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0020
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0017
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0020
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0027
Epoch 9/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0015
Epoch 10/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0014
Epoch 11/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0015
Epoch 12/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0014
Epoch 13/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0014
Epoch 14/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0014
Epoch 15/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0015
Epoc

## GRU

####  Build GRU Model

In [35]:
GRU_model = Sequential()
GRU_model.add(GRU(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100))
GRU_model.add(Dropout(0.2))

GRU_model.add(Dense(units = 1))

#### Compil GRU Model

In [36]:
GRU_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [37]:
GRU_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 60, 300)             │         272,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 60, 300)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 60, 100)             │         120,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 60, 100)             │          60,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 60, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 100)                 │          60,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 514,601 (1.96 MB)

 Trainable params: 514,601 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

#### Fit GRU Model

In [38]:
GRU_History = GRU_model.fit(features_set, labels, epochs = 20, batch_size = 32)

Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0228
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0019
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0016
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0013
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0016
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0014
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0012
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0016
Epoch 9/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0013
Epoch 10/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0011
Epoch 11/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0012
Epoch 12/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0013
Epoch 13/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0011
Epoch 14/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0010
Epoch 15/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0010
Epoc

## Evaluate The Model

#### Evalute Model Training

In [39]:
fig = go.Figure()

# Training loss trace
fig.add_trace(go.Scatter(x=list(range(1, len(RNN_History.history['loss']) + 1)),
                         y=RNN_History.history['loss'],
                         mode='lines',
                         name='RNN Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(LSTM_History.history['loss']) + 1)),
                         y=LSTM_History.history['loss'],
                         mode='lines',
                         name='LSTM Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(GRU_History.history['loss']) + 1)),
                         y=GRU_History.history['loss'],
                         mode='lines',
                         name='GRU Training Loss'))

fig.update_layout(title='Training and Validation Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss',
                  legend_title='Loss Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()

#### Evalute Model Prediction

In [40]:
testing_complete = test
testing_processed = testing_complete.iloc[:, 1:2].values

google_total = pd.concat((train['Close'], testing_complete['Close']), axis=0)

test_inputs = google_total[len(google_total) - len(testing_complete) - 60:].values

In [41]:
len(test)

317

In [42]:
test_inputs = test_inputs.reshape(-1,1)
test_inputs = scaler.transform(test_inputs)

test_features = []
for i in range(60, len(test_inputs)):
    test_features.append(test_inputs[i-60:i, 0])

test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
test_features.shape

(317, 60, 1)

In [43]:
# Evaluate the model on the test set
## RNN
RNN_test_loss = RNN_model.evaluate(test_features_set, test_labels)
print(f'RNN Test Loss: {RNN_test_loss}')

## LSTM
LSTM_test_loss = LSTM_model.evaluate(test_features_set, test_labels)
print(f'LSTM Test Loss: {LSTM_test_loss}')

## GRU
GRU_test_loss = GRU_model.evaluate(test_features_set, test_labels)
print(f'LSTM Test Loss: {GRU_test_loss}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0158
RNN Test Loss: 0.010376575402915478
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0030
LSTM Test Loss: 0.0029588951729238033
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0013
LSTM Test Loss: 0.0014498461969196796


## Make predictions

In [44]:
# RNN
RNN_predictions = RNN_model.predict(test_features_set)
RNN_predictions = scaler.inverse_transform(RNN_predictions)

# LSTM
LSTM_predictions = LSTM_model.predict(test_features_set)
LSTM_predictions = scaler.inverse_transform(LSTM_predictions)

# GRU
GRU_predictions = GRU_model.predict(test_features_set)
GRU_predictions = scaler.inverse_transform(GRU_predictions)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step


In [45]:
len(RNN_predictions)

257

In [46]:
prediction = test['Date'].iloc[60:]
prediction = pd.DataFrame(test.iloc[60:])
prediction['RNN_Open'] = RNN_predictions
prediction['LSTM_Open'] = LSTM_predictions
prediction['GRU_Open'] = GRU_predictions

prediction

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,RNN_Open,LSTM_Open,GRU_Open
2912,2023-12-14 00:00:00-05:00,332.579596,333.426368,327.389431,331.902191,19607300,0.0,0.0,288.057129,321.021729,335.778900
2913,2023-12-15 00:00:00-05:00,330.726635,337.371266,329.959576,333.645508,30001600,0.0,0.0,288.485260,323.041687,336.202057
2914,2023-12-18 00:00:00-05:00,336.195780,346.237408,335.737508,343.308594,18993900,0.0,0.0,288.947449,325.236908,336.790375
2915,2023-12-19 00:00:00-05:00,344.264924,352.254425,343.806683,349.026733,17729400,0.0,0.0,289.533875,327.593781,341.512085
2916,2023-12-20 00:00:00-05:00,347.323229,353.609214,346.466516,347.950836,16369900,0.0,0.0,290.343384,330.307709,347.212494
...,...,...,...,...,...,...,...,...,...,...,...
3164,2024-12-16 00:00:00-05:00,629.979980,631.150024,618.580017,624.239990,10885600,0.5,0.0,609.035400,628.406067,609.541077
3165,2024-12-17 00:00:00-05:00,626.169983,632.369995,616.539978,619.440002,12897800,0.0,0.0,609.211182,629.586853,611.530090
3166,2024-12-18 00:00:00-05:00,621.000000,627.359985,596.320007,597.190002,17075500,0.0,0.0,609.304443,629.640320,610.723694
3167,2024-12-19 00:00:00-05:00,610.390015,611.520020,595.000000,595.570007,14958100,0.0,0.0,609.218262,627.622803,597.502197


In [47]:
len(prediction)

257

In [48]:
fig = go.Figure()

# Actual facebook stock price trace
fig.add_trace(go.Scatter(x=testing_complete.Date,
                         y=testing_complete.Open,
                         mode='lines',
                         name='Actual Facebook Stock Price',
                         line=dict(color='blue')))

# RNN Predicted facebook stock price trace
fig.add_trace(go.Scatter(x=prediction.Date,
                         y=prediction.RNN_Open,
                         mode='lines',
                         name='RNN Predicted Facebook Stock Price',
                         line=dict(color='red')))

# LSTM Predicted facebook stock price trace
fig.add_trace(go.Scatter(x=prediction.Date,
                         y=prediction.LSTM_Open,
                         mode='lines',
                         name='LSTM Predicted Facebook Stock Price',
                         line=dict(color='green')))

# LSTM Predicted facebook stock price trace
fig.add_trace(go.Scatter(x=prediction.Date,
                         y=prediction.GRU_Open,
                         mode='lines',
                         name='GRU Predicted Facebook Stock Price',
                         line=dict(color='yellow')))

fig.update_layout(title='Facebook Stock Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='Facebook Stock Price',
                  legend_title='Price Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()